In [26]:
import pandas as pd
import re
import emoji

df=pd.read_csv("covid19_tweets.csv")

In [27]:
#Clean text
def clean_tweet(text):
    text=text.lower() #lower text
    text=re.sub(r"https?://\S+","",text) #remove links
    text = re.sub(r"www\.\S+", "", text)
    text=re.sub(r"@\w+","",text) #remove mention
    text=re.sub(r"\brt\b","",text) #remove retweet
    text=re.sub(r"#(\w+)",r"\1",text) # remove hashtag
    text=re.sub(r"[\n\t]","",text) # remove newline and tab
    return text.strip()

df["text_clean"]=df["text"].apply(clean_tweet)

def has_emoji(text):
    return any(char in emoji.EMOJI_DATA for char in text)

def extract_emojis(text):
    return [char for char in text if char in emoji.EMOJI_DATA]
df["emojis"]=df["text_clean"].apply(extract_emojis)
df["emoji_count"]=df["emojis"].apply(len)
df["has_emoji"]=df["emoji_count"]>0

#Transform emoji in token
def demojize_text(text):
    text=emoji.demojize(text)
    text=re.sub(r":([a-z_]+):",r" emoji_\1",text)
    return text.strip()
df["text_demojize"]=df["text_clean"].apply(demojize_text)

# List of keywords to identify COVID-19 related tweets
covid_keywords=["covid","covid19","coronavirus","sars-cov-2","pandemic"]

# Function to check if a text mentions COVID-19
def is_covid_related(text):
    pattern = r"\b(" + "|".join(covid_keywords) + r")\b"
    return bool(re.search(pattern, text))

df_with_emoji=df[df["has_emoji"]].copy()
df_with_emoji=df_with_emoji[df_with_emoji["text_clean"].apply(is_covid_related)].copy()

def remove_duplicate_emoji(emoji_list):
    return list(dict.fromkeys(emoji_list))


df_with_emoji["emojis_unique"]=df_with_emoji["emojis"].apply(remove_duplicate_emoji)

df_with_emoji[["text_clean","emojis","emoji_count","emojis_unique","text_demojize"]].sample(100)


,text_clean,emojis,emoji_count,emojis_unique,text_demojize
138806,🙏🕯🙏thoughtsandprayers for his loved ones.covid...,"[🙏, 🕯, 🙏]",3,"[🙏, 🕯]",emoji_folded_hands emoji_candle emoji_folded_h...
137663,just did an interview where they record your a...,"[🥵, 🎤, 😩]",3,"[🥵, 🎤, 😩]",just did an interview where they record your a...
163917,💮🚢🚢🚢🚢 🤔four cargo ships off queensland may hav...,"[💮, 🚢, 🚢, 🚢, 🚢, 🤔]",6,"[💮, 🚢, 🤔]",emoji_white_flower emoji_ship emoji_ship emoji...
33565,after this corona pandemic we need to be added...,[🥺],1,[🥺],after this corona pandemic we need to be added...
158332,📻 we’ve been working with and radio stations ...,"[📻, 👂, 🏾]",3,"[📻, 👂, 🏾]",emoji_radio we’ve been working with and radio...
...,...,...,...,...,...
125424,covid19 update. on august 14 as of 11:00 in ar...,"[✅, ✅]",2,[✅],covid19 update. on august 14 as of 11:00 in ar...
155698,justice baker for deputy ag.👍👏police protectth...,"[👍, 👏]",2,"[👍, 👏]",justice baker for deputy ag. emoji_thumbs_up e...
148908,🖋 sign “authorize covid-19 rapid testing” and ...,"[🖋, 📨]",2,"[🖋, 📨]",emoji_fountain_pen sign “authorize covid-19 ra...
101091,honourable shri sir please 🙏 give us subsidy...,[🙏],1,[🙏],honourable shri sir please emoji_folded_han...


In [42]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import classification_report,accuracy_score

analyzer=SentimentIntensityAnalyzer()

#We generate labels using VADER (Automatically handles emohis)
def label_sentiment(text):
   score=analyzer.polarity_scores(text)['compound']
   if score>0.05:
      return "Positive"
   elif score<=-0.05:
      return "Negative"
   else:
      return "Neutral"
   
df_with_emoji["sentiment"]=df_with_emoji["text_clean"].apply(label_sentiment)

df_with_emoji[["text_clean","emojis","emoji_count","emojis_unique","text_demojize","sentiment"]].sample(1000)

X=df_with_emoji["text_demojize"] #Feature (text containing emoji tokens like 'emoji_face_mask')
y=df_with_emoji["sentiment"] #Target labels

#Training
X_train_raw,X_test_raw,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

#TF-IDF
tfidf=TfidfVectorizer(token_pattern=r"\b\w\w+\b|emoji_\w+",stop_words="english")

#Fit on training data and transform both sets into numerical matrices
X_train_tfidf=tfidf.fit_transform(X_train_raw)
X_test_tfidf=tfidf.transform(X_test_raw)

#Initializing and training Multinomial Naive Bayes Classifier
model=ComplementNB() #Per i dataset sbilanciati
model.fit(X_train_tfidf,y_train)

#Model evaluation
y_pred=model.predict(X_test_tfidf)

print(f"Global Accuracy: {accuracy_score(y_test,y_pred):.2f}\n")
print(classification_report(y_test,y_pred))

Global Accuracy: 0.71

              precision    recall  f1-score   support

    Negative       0.71      0.63      0.67       386
     Neutral       0.73      0.58      0.65       632
    Positive       0.69      0.84      0.76       797

    accuracy                           0.71      1815
   macro avg       0.71      0.69      0.69      1815
weighted avg       0.71      0.71      0.70      1815

